In [1]:
import tiktoken

In [2]:
tokenizer = tiktoken.get_encoding('gpt2')

In [12]:
text = "Hi nithu, How are you doing? <|endoftext|>"
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"},)

In [13]:
print(integers)

[17250, 299, 342, 84, 11, 1374, 389, 345, 1804, 30, 220, 50256]


In [14]:
text_decode = tokenizer.decode(integers)

In [15]:
print(text_decode)

Hi nithu, How are you doing? <|endoftext|>


### Exercise 2.1 

In [17]:
text = "Akwirw ier"
token_id = tokenizer.encode(text)
print(token_id)

[33901, 86, 343, 86, 220, 959]


In [18]:
text_decode = tokenizer.decode(token_id)
print(text_decode)

Akwirw ier


### Building the dataset

In [19]:
with open("the_verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [23]:
enc_sample = enc_text[50:]

In [24]:
# x contains the input tokens and y contains the target tokens which are input tokens shifted by 1
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

In [25]:
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [26]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "------>", desired)

[290] ------> 4920
[290, 4920] ------> 2241
[290, 4920, 2241] ------> 287
[290, 4920, 2241, 287] ------> 257


In [28]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "----->", tokenizer.decode([desired]))

 and ----->  established
 and established ----->  himself
 and established himself ----->  in
 and established himself in ----->  a


In [33]:
## DataLoader and Dataset in pytorch
import torch
from torch.utils.data import DataLoader, Dataset

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids)-max_length, stride):
            input_chunks = token_ids[i:i+max_length]
            target_chunks = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunks))
            self.target_ids.append(torch.tensor(target_chunks))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [51]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=False ,drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    return dataloader

In [52]:
with open("the_verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [53]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [54]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=2, stride=2, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[ 40, 367]]), tensor([[ 367, 2885]])]


In [55]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[2885, 1464]]), tensor([[1464, 1807]])]


In [56]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=8, stride=8, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464, 1807, 3619,  402,  271]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899]])]


In [57]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[10899,  2138,   257,  7026, 15632,   438,  2016,   257]]), tensor([[ 2138,   257,  7026, 15632,   438,  2016,   257,   922]])]


In [58]:
# batch_size = 8
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(f"inputs: \n", inputs)
print(f"targets \n", targets)

inputs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
targets 
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


### Converting token_ids to embeddings

In [59]:
input_ids = torch.tensor([2,3,5,1])

In [60]:
vocab_size = 6
output_dim = 3

In [61]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [62]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [63]:
# applying the embeddings to all the input_ids
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


### Encoding word positions

#### The embeddings in the above are enough to pass it on to the llm's but because of their self-attention mechanism, doesn't have any idea of
#### particular order for the tokens within a sequence, Hence whats happens is that even though the words occur at different position the 
#### embeddings would be still the same. Hence we add positional embeddings.
##### 1) Relative positional embeddings :- this tells how far apart the words tokens are, it helps in generalizability.
##### 2) Absolute positional embeddings :- this just tells where the exact position is and is added to the embeddings.

In [65]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim) 

In [67]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, target = next(data_iter)
print(f"Tokens ID's: \n", inputs)
print(f"Inputs shape: \n", inputs.shape)

Tokens ID's: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Inputs shape: 
 torch.Size([8, 4])


In [68]:
token_embeddings = token_embedding_layer(inputs)

In [69]:
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [74]:
print(token_embeddings[0])

tensor([[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
        [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
        [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
        [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],
       grad_fn=<SelectBackward0>)


In [71]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [75]:
print(pos_embeddings[0])

tensor([-8.1935e-01,  5.5432e-01, -8.2901e-01,  6.4718e-02,  1.2654e-01,
        -5.1345e-02,  1.3903e+00, -5.9934e-01,  5.6520e-01, -4.7247e-01,
         9.8366e-01, -7.2155e-01,  1.0288e+00, -1.1556e-02,  1.2040e+00,
        -1.6929e-02,  8.1006e-01, -2.2914e+00, -1.2456e+00, -3.7353e-01,
        -1.1398e+00, -2.0937e-01, -1.5919e+00,  8.5969e-01, -1.6815e-01,
        -1.2926e-01, -1.2925e+00,  2.9336e+00, -1.0790e+00,  4.7949e-01,
         1.0338e+00,  7.6657e-01, -6.3796e-01,  1.7456e+00, -4.9210e-01,
         2.9416e-01, -1.2237e-01, -4.0603e-01, -6.2785e-01, -1.8055e-02,
        -5.1963e-01,  3.3997e-01,  1.5480e+00,  9.2806e-01, -6.5644e-01,
         1.7713e+00, -1.6067e+00, -8.5211e-01, -2.9453e-01, -8.2847e-02,
         7.2058e-01, -2.9077e-01,  1.4274e+00,  4.5485e-01,  1.1140e+00,
         1.2129e-01, -2.8583e-01, -7.0220e-01, -5.9368e-01,  7.8913e-02,
        -2.4745e-01,  1.3596e+00, -1.6196e+00, -2.5436e-01, -1.0689e+00,
        -3.1881e-01, -4.3838e-01, -6.7342e-01, -5.3

In [73]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


In [77]:
print(input_embeddings[0])

tensor([[-0.3281,  1.6782,  0.6298,  ..., -0.2670, -1.6620,  0.2165],
        [ 0.8674, -0.6925, -0.6063,  ...,  1.2927,  0.5018, -0.6181],
        [-0.4869, -1.7733, -0.3802,  ...,  2.0836,  2.6533,  0.9384],
        [-0.0091,  1.3356,  1.8771,  ..., -1.8233,  0.9045,  1.6972]],
       grad_fn=<SelectBackward0>)
